# Création des tables Python + SQLite

In [10]:
import sqlite3

# Création d'une base en RAM
conn = sqlite3.connect(":memory:")
cur = conn.cursor()


## CLIENTS 

In [11]:
cur.execute("""
CREATE TABLE clients (
    client_id INTEGER PRIMARY KEY,
    nom TEXT,
    ville TEXT
)
""")


## COMPTES 

In [12]:
cur.execute("""
CREATE TABLE comptes (
    compte_id INTEGER PRIMARY KEY,
    client_id INTEGER,
    type_compte TEXT,
    solde REAL,
    FOREIGN KEY (client_id) REFERENCES clients(client_id)
)
""")


## TRANSACTIONS

In [13]:
cur.execute("""
CREATE TABLE transactions (
    transaction_id INTEGER PRIMARY KEY,
    compte_id INTEGER,
    montant REAL,
    date TEXT,
    FOREIGN KEY (compte_id) REFERENCES comptes(compte_id)
)
""")


## CONSEILLERS

In [14]:
cur.execute("""
CREATE TABLE conseillers (
    conseiller_id INTEGER PRIMARY KEY,
    nom TEXT
)
""")


## AFFECTATIONS

In [15]:
cur.execute("""
CREATE TABLE affectations (
    client_id INTEGER,
    conseiller_id INTEGER,
    FOREIGN KEY (client_id) REFERENCES clients(client_id),
    FOREIGN KEY (conseiller_id) REFERENCES conseillers(conseiller_id)
)
""")

In [16]:
cur.executemany("INSERT INTO clients VALUES (?, ?, ?)", [
    (1, "Alice", "Paris"),
    (2, "Bob", "Lyon"),
    (3, "Charlie", "Paris"),
    (4, "Diana", "Marseille")
])

cur.executemany("INSERT INTO comptes VALUES (?, ?, ?, ?)", [
    (101, 1, "Courant", 2500),
    (102, 1, "Epargne", 8000),
    (103, 2, "Courant", 1200),
    (104, 3, "Courant", 500),
])

cur.executemany("INSERT INTO transactions VALUES (?, ?, ?, ?)", [
    (1, 101, -200, "2024-01-10"),
    (2, 101, 500, "2024-01-12"),
    (3, 103, -100, "2024-01-15"),
    (4, 104, -50, "2024-01-20")
])

cur.executemany("INSERT INTO conseillers VALUES (?, ?)", [
    (1, "Martin"),
    (2, "Sophie")
])

cur.executemany("INSERT INTO affectations VALUES (?, ?)", [
    (1, 1),
    (2, 1),
    (3, 2)
])

conn.commit()


## EXO 1 : Afficher les clients avec un compte 

In [19]:
cur.execute("select nom, type_compte, solde from clients inner join comptes on comptes.client_id=clients.client_id")
rows = cur.fetchall()

for row in rows:
    print(row)


('Alice', 'Courant', 2500.0)
('Alice', 'Epargne', 8000.0)
('Bob', 'Courant', 1200.0)
('Charlie', 'Courant', 500.0)


## EXO 2 : Afficher tous les clients même ceux sans compte 

In [25]:
cur.execute("select nom, type_compte from clients left join comptes on comptes.client_id=clients.client_id")
rows = cur.fetchall() 

for row in rows:
    print(row)


('Alice', 'Courant')
('Alice', 'Epargne')
('Bob', 'Courant')
('Charlie', 'Courant')
('Diana', None)


## EXO 3 : FULL join (non interprété par SQLite)
FULL JOIN = LEFT + RIGHT join (right join non interprété par SQLite) <br>
Donc on fait un double left joni en changeant juste ordre des tables 

In [28]:
cur.execute(""" SELECT clients.client_id, clients.nom, comptes.type_compte, comptes.solde 
FROM clients LEFT JOIN comptes ON clients.client_id = comptes.client_id 
UNION 
SELECT clients.client_id, clients.nom, comptes.type_compte, comptes.solde 
FROM comptes LEFT JOIN clients ON comptes.client_id = clients.client_id """)
rows = cur.fetchall()
for row in rows : 
    print(row)


(1, 'Alice', 'Courant', 2500.0)
(1, 'Alice', 'Epargne', 8000.0)
(2, 'Bob', 'Courant', 1200.0)
(3, 'Charlie', 'Courant', 500.0)
(4, 'Diana', None, None)


## EXO 4 : CROSS JOIN 

In [29]:
cur.execute("""
SELECT clients.nom AS client, conseillers.nom AS conseiller
FROM clients
CROSS JOIN conseillers
""")
rows = cur.fetchall()
for row in rows : 
    print(row)

('Alice', 'Martin')
('Alice', 'Sophie')
('Bob', 'Martin')
('Bob', 'Sophie')
('Charlie', 'Martin')
('Charlie', 'Sophie')
('Diana', 'Martin')
('Diana', 'Sophie')


## EXO 5 : SELF JOIN - comparer les clients entre eux

In [31]:
cur.execute("""
select 
    c1.nom as client_1,
    c2.nom as client_2,
    c1.ville
from clients c1
join clients c2
    ON c1.ville = c2.ville
   and c1.client_id < c2.client_id
""")
rows = cur.fetchall()
for row in rows : 
    print(row)

('Alice', 'Charlie', 'Paris')
